# トピックへの自動ラベリング


## コンテンツを取得

In [1]:
import requests
from bs4 import BeautifulSoup

# url = "https://ledge.ai/authorinterview-book-bert-int/"
# url = "https://ainow.ai/2023/01/10/271486/"
# url = "https://ja.stateofaiguides.com/20230105-cramming-bert/"
url = "https://www.nowhere.co.jp/blog/archives/20160524-14722.html"
# url = "https://www.yomiuri.co.jp/world/20230104-OYT1T50056/"
# url = "https://www3.nhk.or.jp/kansai-news/20230104/2000069636.html"
# url = "https://news.yahoo.co.jp/articles/fad0c4f41d46b686e0566bf10e4c016a641a9dab"
# url = "https://news.yahoo.co.jp/articles/4d2d14fd1ca1dc9b134c5f493896a7e30fc1e781"
res = requests.get(url)

soup = BeautifulSoup(res.content, "lxml")
for tg in ["script", "noscript", "meta"]:
    try:
        soup.find(tg).replace_with(" ")
    except:
        pass
soup.get_text()[:200]

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


'\n\n \n\n\n\n\n\n\n   Google発 世界最高精度の構文解析器 “SyntaxNet” はどこがすごいのか - Eyes, JAPAN Blog\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Eyes, JAPAN Blog\n          \n\n            1'

### 簡易クレンジング

In [2]:
import re

def clean_text(text: str):
    contents = []
    for txt in re.split(r"(。|\n)", text):
        txt = txt.strip().replace("\u200b", "").replace("\u3000", " ")
        txt = re.sub(r"\n+", "\n", txt)
        txt = re.sub(r"([\W])\1+", " ", txt)
        if not txt:
            continue
        # contents.append(txt)
        # contents.append(txt.split("\n")[-1])
        contents.extend(txt.split("\n"))
    return contents

text = soup.get_text()
contents = clean_text(text)
contents[:5]

['Google発 世界最高精度の構文解析器 “SyntaxNet” はどこがすごいのか - Eyes, JAPAN Blog',
 'Eyes, JAPAN Blog',
 '1997年からほぼ毎日更新！会津大学発のベンチャー企業、株式会社Eyes, JAPANによるBlog',
 '。',
 'コンピュータ、ネットワーク、Hi-Tech Gadget、魔法の様な技術などのGeekなネタから会津のローカルネタまで']

In [3]:
# # pickup wiki data to analyze topic
# wdb = WikiDb(mode="train")
# records = wdb.select_by_document_id(document_ids=[
#     "Doc01GN7P2K02Y2FC8BXHMZ5V1A34",    # 南部煎餅
#     "Doc01GN7P3C1ZPVGB8AXJC75M9ZQ9",    # 龍が如く OF THE END
#     "Doc01GN7P3C0YPAWH4KT16P7NNQBK",    # 自動運転車
#     "Doc01GN7P3BS2JNYS3HYV0RM1TT7T",    # デュケイン大学
#     "Doc01GN7P3BK88YCXKS64H6G2HK0Y",    # 深蒸し茶
#     ])
# X: TextSequences = [WikiRecord(*rec).paragraph.splitlines() for rec in records]
# pipe_topic.fit(X)

## トピックモデルのパイプラインを構築

In [4]:
import joblib

from app.component.models.model import TextSequences
from app.component.models.pipeline import Pipeline
from app.component.models.vectorizer import VectorizerBoW, VectorizerWord2vec
from app.domain.models.tokenizer import TokenizerWord
from app.domain.models.topic_model import TopicModel
from app.infra.wikidb import WikiDb, WikiRecord

In [5]:
pipe_topic = Pipeline(
    steps=[
        (TokenizerWord(use_stoppoes=True), None),
        (VectorizerBoW(), None),
        (TopicModel(n_topics=10, n_epoch=2000), None),
    ],
    name="pipe_topic_sample",
    do_print=True,
)


In [6]:
# X: TextSequences = [[snt] for snt in contents]
X: TextSequences = [contents]
# pipe_topic[:1].fit(X).transform(X)        # for debugging
pipe_topic.fit(X)

2023/01/15 22:26:47 INFO Start to fit n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/15 22:26:47 INFO End to fit n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/15 22:26:47 INFO Start to transform n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/15 22:26:47 INFO End to transform n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/15 22:26:47 INFO Start to fit n_step=1 model=VectorizerBoW()
2023/01/15 22:26:47 INFO End to fit n_step=1 model=VectorizerBoW()
2023/01/15 22:26:47 INFO Start to transform n_step=1 model=VectorizerBoW()
2023/01/15 22:26:47 INFO End to transform n_step=1 model=VectorizerBoW()
2023/01/15 22:26:47 INFO Start to fit n_step=2 model=TopicModel(n_topics=10, n_epoch=2000, random_state=RandomState(MT19937))
2023/01/15 22:26:47 INFO End to fit n_step=2 model=TopicModel(n_topics=10, n_epoch=2000, rando

Pipeline(steps=[(JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False), None), (VectorizerBoW(), None), (TopicModel(n_topics=10, n_epoch=2000, random_state=RandomState(MT19937)), None)], name=pipe_topic_sample, do_print=True, args=(), kwargs={})

In [7]:
model_bow: VectorizerBoW = pipe_topic.get_model(1)
model_topic: TopicModel = pipe_topic.get_model(-1)
model_bow, model_topic


(VectorizerBoW(),
 TopicModel(n_topics=10, n_epoch=2000, random_state=RandomState(MT19937)))

## Word2Vec モデルをロード

In [8]:
# # use original pretrained newsvec model (with only noun and verb tokens)
pipe_newsvec = joblib.load("data/pipe_newswikivec.gz")
# pipe_newsvec = joblib.load("data/pipe_newsvec.gz")
pipe_newsvec
model_vectorizer: VectorizerWord2vec = pipe_newsvec.get_model(-1)
model_vectorizer
w2v = model_vectorizer.model

In [9]:
# # use original pretrained wikivec model (with only noun and verb tokens)
# pipe_wikivec = joblib.load("data/pipe_wikivec.noun-verb.gz")
# pipe_wikivec
# model_vectorizer: VectorizerWord2vec = pipe_wikivec.get_model(-1)
# model_vectorizer
# w2v = model_vectorizer.model

In [10]:
# # use shiroyagi's word2vec pretrained model
# from gensim.models.word2vec import Word2Vec
# w2v = Word2Vec.load("data/word2vec.gensim.model")
# w2v

In [11]:
len(w2v.wv.key_to_index)

569888

In [12]:
# 各トピックの確率の合計が 1 になることを確認しておく
model_topic.get_topic_probabilities().sum(-1)

array([0.99999994, 1.0000001 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ],
      dtype=float32)

In [13]:
def pickup_topic_words(model_topic: TopicModel, topn: int = -1):
    # topn: 各トピックの上位の単語の数
    topics = []
    for topic_probs in model_topic.get_topic_probabilities():
        indices = topic_probs.argsort()[::-1][:topn]
        topic = [(model_bow.vocab[idx], topic_probs[idx]) for idx in indices]
        topics.append(topic)
    return topics


In [14]:
pickup_topic_words(model_topic, topn=30)[0][:5]


[('June', 0.0016529982),
 ('February', 0.0016529635),
 ('August', 0.0016529604),
 ('July', 0.0016529509),
 ('SyntaxNet', 0.0016529479)]

In [15]:
def parse_topic(topics: list):
    words = []
    numbs = []
    for w, p in topics:
        do_skip: bool = False
        do_skip |= bool(re.search(r"^[あ-ん]", w))    # です、ます などは、スキップ
        do_skip |= bool(re.search(r"[あ-ん]$", w))    # 動名詞 などは、スキップ
        do_skip |= w[0].isnumeric()                  # 数字から始まるラベルはスキップ
        if do_skip:
            continue
        words.append(w)
        numbs.append(p)
    return words, numbs


In [16]:
import numpy
import re


def estimate_topic_label(w2v, topics: list):
    proper_topics = {}
    topic_labels = []

    for idx_tpc, tpc in enumerate(topics):
        # トピック情報を、単語リストとその確率リストに分解する
        _words, _probs = parse_topic(tpc)

        # word2vec モデルに含まれる単語のみに絞る
        words = [w for w in _words if w in w2v.wv]
        probs = [p for w, p in zip(_words, _probs) if w in w2v.wv]

        # 単語リストからベクトルに変換し、期待値ベクトルを算出
        vectors = w2v.wv[words]
        probs = numpy.array(probs).reshape(-1, 1)
        topic_vector = (vectors * probs).sum(axis=0)    # 期待値ベクトル

        # トピックに対する期待値ベクトルに類似するベクトルを十分な数(topn=100) を取得しておく
        estimated_topic_labels = w2v.wv.similar_by_vector(topic_vector, topn=100)

        # ラベルと類似度を取得し、最も類似度が高い最初のインデックスの要素を保持
        labels, similarities = parse_topic(estimated_topic_labels)
        topic_label = labels[0]
        similarity = similarities[0]

        # 重複しないトピックラベル集合(proper_topics)として記録しておく
        if topic_label not in proper_topics:
            proper_topics[topic_label] = (idx_tpc, similarity, words, probs)

        # 重複を許すトピックラベル(topic_labels)として記録しておく
        topic_labels.append((topic_label, similarity))

    return topic_labels, proper_topics


In [17]:
# トピック数を自動で特定するサンプル
# # トピック数が proper_topics と一致するまで、減らしていくことで、トピック数を特定する

# w2v : is already loaded
topic_label_counter = {}

n_topic_words = 17      # to calculate the average over
n_topics = 10            # default topic numbers


rs = numpy.random.RandomState(12345)

while True:
    # トピックモデルのパイプラインを構築
    pipe_topic = Pipeline(
        steps=[
            (TokenizerWord(use_stoppoes=True, use_orgform=True), None),
            (VectorizerBoW(), None),
            (TopicModel(n_topics=n_topics, n_epoch=2000, random_state=rs), None),
        ],
        name="pipe_topic",
        do_print=False,
    )

    # トピックモデルを学習
    X: TextSequences = [contents]
    pipe_topic.fit(X)

    model_topic: TopicModel = pipe_topic.get_model(-1)
    topics = pickup_topic_words(model_topic, topn=n_topic_words)
    topic_labels, proper_topics = estimate_topic_label(w2v, topics)

    # トピックラベルをカウント
    for tpc in proper_topics:
        cnt = topic_label_counter.get(tpc, 0) + 1
        topic_label_counter[tpc] = cnt
    
    # ループの終了条件
    if len(proper_topics) >= n_topics:
        break

    # 状態/処理文脈としてトピック数を更新・保持
    n_topics = len(proper_topics)

In [18]:
# トピックの出力
# # 自動付与したラベルと、各トピックの上位単語の表示
# # 理想的には、この出力結果に違和感がないこと

for idx_tpc, (lbl, sim) in enumerate(topic_labels):
    print("-" * 100)
    print(f"topic[{idx_tpc}]: {lbl} : {topic_label_counter[lbl]} ({sim:0.3f})")
    print(" " * 4 + f" ... {[_t for _t, _s in topics[idx_tpc][:20]]}")


----------------------------------------------------------------------------------------------------
topic[0]: FortniteintoPUBGwithDeep : 2 (0.832)
     ... ['Processing', 'Apollo', 'ご', 'T', 'Boaty', 'Land', 'Language', 'Lisp', 'Prototyping', 'Git', 'NLU', 'Natural', 'UA', 'at', 'F', '2022', 'gRPC']
----------------------------------------------------------------------------------------------------
topic[1]: inmultipleenvironments : 2 (0.833)
     ... ['ご', 'F', 'UA', 'Natural', 'Git', 'Lisp', 'Land', 'NLU', 'Prototyping', 'Boaty', 'T', 'それほど', 'Language', '2022', 'Apollo', 'at', 'Processing']


In [19]:
# 実際に期待値ベクトルを算出するときに使った単語を表示
# # 上記の自動付与ラベルと上位単語の関係性に違和感があるときに確認すると良いだろう
print("estimated n_topic:", len(proper_topics))

for lbl, v in proper_topics.items():
    tpc_idx, sim, words, probs = v
    print(f"[{tpc_idx:02d}]: {lbl}: {words}")


estimated n_topic: 2
[00]: FortniteintoPUBGwithDeep: ['Processing', 'Apollo', 'T', 'Land', 'Language', 'Lisp', 'Git', 'NLU', 'Natural', 'UA', 'at', 'F']
[01]: inmultipleenvironments: ['F', 'UA', 'Natural', 'Git', 'Lisp', 'Land', 'NLU', 'T', 'Language', 'Apollo', 'at', 'Processing']


In [20]:
for lbl, cnt in topic_label_counter.items():
    print(f"{lbl} : {cnt}")

FortniteintoPUBGwithDeep : 2
inmultipleenvironments : 2


In [21]:
print(" ".join(contents)[:200])

Google発 世界最高精度の構文解析器 “SyntaxNet” はどこがすごいのか - Eyes, JAPAN Blog Eyes, JAPAN Blog 1997年からほぼ毎日更新！会津大学発のベンチャー企業、株式会社Eyes, JAPANによるBlog 。 コンピュータ、ネットワーク、Hi-Tech Gadget、魔法の様な技術などのGeekなネタから会津のローカルネタまで 。 Eyes, 


## LDA の可視化

In [22]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()


/usr/local/lib/python3.10/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [23]:
model_bow: VectorizerBoW = pipe_topic.get_model(1)
model_topic: TopicModel = pipe_topic.get_model(-1)

lda = model_topic.model
bow = pipe_topic[:-1](X)

vis = pyLDAvis.gensim_models.prepare(lda, corpus=bow, dictionary=model_bow.vocab)
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.10/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  fr

In [24]:
len(model_bow.vocab)

586

In [25]:
numpy.array(bow).shape

(1, 586, 2)